## 68. Згорткові нейромережі. Tensorboard

### Завдання:

#### Створіть модель згорткової нейронної мережі (CNN) із трьома шарами. Підключіть Tensorboard, та відобразіть в ньому зміну розподілу ваг моделі, обчислювальний граф моделі та зміну її метрик.

In [1]:
import torch
print(torch.__version__)

2.0.0


In [2]:
import torch
import torch.nn as nn
import torch.optim as opt
torch.set_printoptions(linewidth=120)
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter

Це визначення класу в PyTorch для моделі згорткової нейронної мережі (CNN). Клас успадковує від класу nn.Module, який є базовим класом для всіх модулів нейронних мереж в PyTorch.

Метод конструктора класу (init) ініціалізує шари CNN, які включають два згорткові шари (conv1 і conv2), за якими йдуть три повнозв'язні (dense) шари (fc1, fc2 і fc3) та вихідний шар (out). Згорткові шари використовують двовимірну згортку для вивчення просторових особливостей вхідних даних, а повнозв'язні шари використовують лінійні перетворення для створення високорівневих представлень вхідних даних.

Метод forward реалізує прохід вперед CNN, який приймає тензор вхідних даних x як аргумент і послідовно застосовує згорткові та повнозв'язні шари для обчислення тензора вихідних даних. Тензор вихідних даних має форму (batch_size, num_classes), де num_classes - кількість вихідних класів для задачі класифікації.

Метод forward застосовує функцію активації ReLU до виходу кожного шару, що додає нелінійність до моделі та дозволяє їй вивчати більш складні особливості. Також він використовує максимальне пулінгування, щоб зменшити розмір карт ознак між згортковими шарами, що допомагає зменшити кількість параметрів моделі та запобігти перенавчанню. Нарешті, тензор вихідних даних передається через вихідний шар, який застосовує лінійне перетворення, щоб отримати кінцеві оцінки класифікації.

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)

        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.fc3 = nn.Linear(in_features=60, out_features=30)
        self.out = nn.Linear(in_features=30, out_features=10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2)
        x = torch.flatten(x,start_dim = 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)

        return x

In [4]:
# Завантажуємо дані 
train_set = torchvision.datasets.FashionMNIST(root="./data",
train = True,
 download=True,
transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(train_set,
                                           batch_size = 100,
                                           shuffle = True)

In [5]:
# Створюємо об'єкт класу SummaryWriter, який дозволяє записувати і відображати результати 
# візуалізації даних та метрик під час навчання моделі в TensorBoard.
tb = SummaryWriter()
model = CNN()
images, labels = next(iter(train_loader))
grid = torchvision.utils.make_grid(images)
tb.add_image("images", grid)
tb.add_graph(model, images)
tb.close()

In [6]:
model = CNN()
train_loader = torch.utils.data.DataLoader(train_set,batch_size = 100, shuffle = True)
optimizer = opt.Adam(model.parameters(), lr= 0.01)
criterion = torch.nn.CrossEntropyLoss()

tb = SummaryWriter()

def get_num_correct(preds, labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

for epoch in range(10):

    total_loss = 0
    total_correct = 0

    for images, labels in train_loader:
        images, labels = images, labels
        preds = model(images)

        loss = criterion(preds, labels)
        total_loss+= loss.item()
        total_correct+= get_num_correct(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tb.add_scalar("Loss", total_loss, epoch)
    tb.add_scalar("Correct", total_correct, epoch)
    tb.add_scalar("Accuracy", total_correct/ len(train_set), epoch)

    tb.add_histogram("conv1.bias", model.conv1.bias, epoch)
    tb.add_histogram("conv1.weight", model.conv1.weight, epoch)
    tb.add_histogram("conv2.bias", model.conv2.bias, epoch)
    tb.add_histogram("conv2.weight", model.conv2.weight, epoch)
    tb.add_histogram("fc1.bias", model.fc1.bias, epoch)
    tb.add_histogram("fc1.weight", model.fc1.weight, epoch)
    tb.add_histogram("fc2.bias", model.fc2.bias, epoch)
    tb.add_histogram("fc2.weight", model.fc2.weight, epoch)
    tb.add_histogram("fc3.bias", model.fc3.bias, epoch)
    tb.add_histogram("fc3.weight", model.fc3.weight, epoch)

    print("epoch:", epoch, "total_correct:", total_correct, "loss:",total_loss)

tb.close()

epoch: 0 total_correct: 45761 loss: 372.2201583981514
epoch: 1 total_correct: 51013 loss: 244.6232704371214
epoch: 2 total_correct: 51700 loss: 225.011220946908
epoch: 3 total_correct: 52141 loss: 212.66279566287994
epoch: 4 total_correct: 52402 loss: 204.64929619431496
epoch: 5 total_correct: 52609 loss: 199.54695862531662
epoch: 6 total_correct: 52711 loss: 200.36565117537975
epoch: 7 total_correct: 52824 loss: 193.8617832660675
epoch: 8 total_correct: 52932 loss: 195.5598203241825
epoch: 9 total_correct: 53047 loss: 189.591712936759


In [ ]:
! tensorboard --logdir runs


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

E0419 17:25:12.580253 4375436736 application.py:125] Failed to load plugin WhatIfToolPluginLoader.load; ignoring it.
Traceback (most recent call last):
  File "/Users/mac/Desktop/Sturtup_Academy/venv/lib/python3.11/site-packages/tensorboard/backend/application.py", line 123, in TensorBoardWSGIApp
    plugin = loader.load(context)
             ^^^^^^^^^^^^^^^^^^^^
  File "/Users/mac/Desktop/Sturtup_Academy/venv/lib/python3.11/site-packages/tensorboard_plugin_wit/wit_plugin_loader.py", line 57, in load
    from tensorboard_plugin_wit.wit_plugin import WhatIfToolPlugin
  File "/Users/mac/Desktop/Sturtup_Academy/venv/lib/python3.11/site-packages/tensorboard_plugin_wit/wit_plugin.py", line 40, in <module>
    from tensorboard_plugin_wit._utils import common_utils
  File "/Users/mac/Desktop/Sturtup_Ac

![Alt-текст](1.JPG "Picture")

![Alt-текст](2.JPG "Picture")

In [5]:
from IPython.display import Image
Image(url='https://www.dropbox.com/s/7njrwyk8pgd4swk/1.jpg?dl=0')

In [6]:
from IPython.display import Image
Image(url='https://drive.google.com/file/d/1LbzSdsW5hJT-mF_VSdWlBJ3B8udbulX9/view?usp=sharing')